In [1]:
import os
import sys
import pandas
import hashlib
import hmac
import requests
import datetime
import time
import json
from tqdm import tqdm
import configparser

In [2]:
# APIアカウント
config = configparser.ConfigParser()
config.read('./config.ini')

BF_ACCESSKEY = config['BITFLYER']['ACCESSKEY']
BF_SECRETKEY = config['BITFLYER']['SECRETKEY']



In [3]:
response = requests.get("https://api.bitflyer.jp/v1/getboard/")
print(response.json())

{'mid_price': 1239323.0, 'bids': [{'price': 1238995.0, 'size': 0.05}, {'price': 1238964.0, 'size': 0.0992}, {'price': 1238963.0, 'size': 0.35}, {'price': 1238610.0, 'size': 0.2983}, {'price': 1238570.0, 'size': 0.15}, {'price': 1238280.0, 'size': 0.2}, {'price': 1238227.0, 'size': 0.31}, {'price': 1238141.0, 'size': 0.3312859}, {'price': 1238127.0, 'size': 1.0}, {'price': 1238015.0, 'size': 0.204}, {'price': 1237880.0, 'size': 0.42}, {'price': 1237668.0, 'size': 0.206}, {'price': 1237612.0, 'size': 0.17}, {'price': 1237610.0, 'size': 1.1}, {'price': 1237225.0, 'size': 1.19825}, {'price': 1237170.0, 'size': 0.8}, {'price': 1236740.0, 'size': 1.3309845}, {'price': 1236738.0, 'size': 0.198}, {'price': 1236659.0, 'size': 3e-05}, {'price': 1236367.0, 'size': 0.01}, {'price': 1236250.0, 'size': 0.035}, {'price': 1236045.0, 'size': 0.308}, {'price': 1236034.0, 'size': 0.005}, {'price': 1236000.0, 'size': 0.1100002}, {'price': 1235878.0, 'size': 0.30000013}, {'price': 1235877.0, 'size': 0.8}, 

In [4]:
base_url = "https://api.bitflyer.jp"
path_url = "/v1/ticker?product_code=BTC_JPY"
method = "GET"

timestamp = str(datetime.datetime.today())

param = {
    "product_code" : "FX_BTC_JPY"
}
body = json.dumps(param)

message = timestamp + method + path_url + body
signature = hmac.new(bytearray(BF_SECRETKEY.encode('utf-8')), message.encode('utf-8') , digestmod = hashlib.sha256 ).hexdigest()

headers = {
    'ACCESS-KEY' : BF_ACCESSKEY,
    'ACCESS-TIMESTAMP' : timestamp,
    'ACCESS-SIGN' : signature,
    'Content-Type' : 'application/json'
}

response = requests.get( base_url + path_url , data = body , headers = headers)

In [5]:
def bf_checkLTP():
    return requests.get("https://api.bitflyer.jp/v1/ticker?product_code=FX_BTC_JPY")
def cc_checkLTP():
    return requests.get("https://coincheck.com/api/ticker")

In [6]:
gap_hist=[]
status = {
    "position": "none",
    "sell_gap": -1
}

with open('output_bffx-cc_0809.csv', mode='w') as f:
    while True:
        bf_ltp = int(json.loads(bf_checkLTP().text)['ltp'])
        cc_ltp = int(json.loads(cc_checkLTP().text)['last'])
        dd = datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")
        
        gap_hist.insert(0, bf_ltp-cc_ltp)
        gap_hist = gap_hist[:400] # 1分20回
        gap_ave = int(sum(gap_hist)/len(gap_hist))
        f.write("{}, {}, {}, {}, {}\n".format(dd, bf_ltp, cc_ltp, bf_ltp-cc_ltp, gap_ave))
        f.flush()
        
        if ((bf_ltp-cc_ltp) - gap_ave >= 1500) and status['position'] == "none":
            print("{}, {}, {}, {}, {}, {}".format(dd, bf_ltp, cc_ltp, bf_ltp-cc_ltp, gap_ave, (bf_ltp-cc_ltp) - gap_ave))
            print("SELL, BFFX, {}BTC, {}".format(0.1, bf_ltp))
            print("BUY, CC, {}BTC, {}".format(0.1, cc_ltp))
            status['position'] = "sell"
            status['sell_gap'] = (bf_ltp-cc_ltp)
        if (((bf_ltp-cc_ltp) - gap_ave <= -1000) or ((status['sell_gap'] - (bf_ltp-cc_ltp))>=3000) ) and status['position'] == "sell":
            print("{}, {}, {}, {}, {}, {}".format(dd, bf_ltp, cc_ltp, bf_ltp-cc_ltp, gap_ave, (bf_ltp-cc_ltp) - gap_ave))
            print("BUY, BFFX, {}BTC, {}".format(0.1, bf_ltp))
            print("SELL, CC, {}BTC, {}".format(0.1, cc_ltp))
            print("--------------------")
            status['position'] = "none"
        
        time.sleep(3)

2020/08/09 06:44:25, 1259607, 1247018, 12589, 10097, 2492
SELL, BFFX, 0.1BTC, 1259607
BUY, CC, 0.1BTC, 1247018
2020/08/09 07:48:07, 1254198, 1244289, 9909, 11327, -1418
BUY, BFFX, 0.1BTC, 1254198
SELL, CC, 0.1BTC, 1244289
--------------------
2020/08/09 08:15:51, 1250834, 1238095, 12739, 11175, 1564
SELL, BFFX, 0.1BTC, 1250834
BUY, CC, 0.1BTC, 1238095
2020/08/09 08:23:17, 1252268, 1242234, 10034, 11039, -1005
BUY, BFFX, 0.1BTC, 1252268
SELL, CC, 0.1BTC, 1242234
--------------------


KeyboardInterrupt: 